In [1]:
import requests
import json
import os
import re
import csv
import time

from urllib.request import urlopen
from urllib.error import HTTPError
from time import sleep
from bs4 import BeautifulSoup

In [70]:
# ESPN_IN_URL = "https://site.web.api.espn.com/apis/site/v2/sports/cricket/1426547/playbyplay?contentorigin=espn&event=1426557&page=1&period=1"

# response = requests.get(ESPN_IN_URL, timeout=100)
# data = response.json()
# max_page_no = data['commentary']['pageCount']
# os.makedirs("espn.in", exist_ok=True)
# with open("espn.in/1426557.json", 'w') as f:
#     json.dump(data, f, indent=4)
# data

In [71]:
# season_no = '2021'

In [72]:
def get_soup_from_url(url):
    # Helper to get response soup from url
    sleep(1)
    try:
        html = urlopen(url).read()
    except HTTPError:
        print("Link Cannot be Reached")
        return -1
    #soup = BeautifulSoup(html,"lxml")
    soup = BeautifulSoup(html, "html.parser")
    return soup


In [73]:
def get_all_matchids_for_season(season, format):
    #season = "2018"
    #season = "2018/19"
    season = season.replace('/', '%2F')
    url = "http://www.espncricinfo.com/ci/engine/series/index.html?season={};view=season".format(season)
    soup = get_soup_from_url(url)
    d={}
    for section in soup.find_all('div', class_='match-section-head'):
        # print(section.text)
        if(section.text == format):
            
            series_in_section = [series['data-series-id']
                                for series in section.find_next(
                                    'section',
                                    class_='series-summary-wrap'
                                ).find_all(
                                    'section',
                                    class_="series-summary-block collapsed"
                                )
                                ]
            series_in_section = [
                get_soup_from_url('http://www.espncricinfo.com/ci/engine/match/index/series.html?series={}'.format(s)) 
                for s in series_in_section
            ]
            matches_in_section = [
                re.findall('\/([0-9]+)\/', 
                        m.find('a')['href'])[1] 
                for ss in series_in_section 
                for m in ss.find_all(class_='match-no')
            ]
            d[section.text] = matches_in_section
            
    return d

In [74]:
def match_id_list(season_no):
    match_ids = []
    match_ids = get_all_matchids_for_season(season_no, "Tests")
    # match_ids['Tests']

    season_str = season_no.replace("/", "_")

    os.makedirs(f"./Tests", exist_ok=True)
    with open(f"./Tests/{season_str}_mid.txt", 'w') as f:
        json.dump(match_ids, f)

In [75]:
# Tests, One-Day Internationals, Twenty20 Internationals

In [ ]:
start_year = 2005
end_year = 2024

for year in range(start_year, end_year + 1):
    if year < end_year:
        match_id_list(f"{year}/{str(year + 1)[-2:]}")
    match_id_list(f"{year}")


In [37]:
# season_str = season_no.replace("/", "_")

# with open(f"./{season_str}_mid.txt", 'w') as f:
#     json.dump(match_ids, f)

In [ ]:
folder_path = "Tests"

# Use a set to keep only unique numbers
unique_numbers = set()

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        with open(os.path.join(folder_path, filename), 'r') as file:
            # Load JSON content from the file
            data = json.load(file)
            # Extract numbers and add them to the set
            unique_numbers.update(data.get("Tests", []))

# Sort the unique numbers in ascending order
sorted_numbers = sorted(unique_numbers, key=int)

# Write the combined data to a new file
with open("all_tests.txt", "w") as output_file:
    for number in sorted_numbers:
        output_file.write(number + "\n")

In [85]:
def get_series_id_from_match_id(match_id_list):

    matchid_seriesid_set = set()
    
    for i in match_id_list:
        original_url = f"https://www.espncricinfo.com/matches/engine/match/{i}.html"
        
        # Perform a HEAD request to get the redirected URL
        response = requests.head(original_url, allow_redirects=True)
        redirected_url = response.url
        match = re.search(r'/series/.+?-(\d+)/', redirected_url)

        if match:
            series_id = match.group(1)
            matchid_seriesid_set.add((i, series_id))
    
    return matchid_seriesid_set
            # print(series_id)

In [86]:
# season_str = season_no.replace("/", "_")
# with open(f"./Tests/{season_str}_mid.txt", 'r') as f:
#     match_ids = json.load(f)

# match_ids_test = match_ids['Tests']
# mid_sid = get_series_id_from_match_id(match_ids_test)
# mid_sid = list(mid_sid)

# os.makedirs(f"./Tests", exist_ok=True)
# with open(f"./Tests/{season_str}_mid_sid.txt", 'w') as f:
#     json.dump(mid_sid, f)

In [6]:
# # season_str = season_no.replace("/", "_")
# match_ids = []
# with open("all_tests.txt", 'r') as f:
#     match_ids = json.load(f)

# print(match_ids)
# break
# # match_ids_test = match_ids['Tests']
# mid_sid = get_series_id_from_match_id(match)
# mid_sid = list(mid_sid)

# os.makedirs(f"./Tests", exist_ok=True)
# with open(f"./Tests/{season_str}_mid_sid.txt", 'w') as f:
#     json.dump(mid_sid, f)

In [27]:
# match_id = 1426557

def espn_in_comms_data(mid_sid_list):

    for match_id, series_id in mid_sid_list:

        # time.sleep(2)

        match_data = []
        for inn_no in range(1, 5):

            ESPN_IN_URL = f"https://site.web.api.espn.com/apis/site/v2/sports/cricket/{series_id}/playbyplay?contentorigin=espn&event={match_id}&page=1&period={inn_no}"

            response = requests.get(ESPN_IN_URL, timeout=100)
            data = response.json()
            max_page_no = data['commentary']['pageCount']

            if(max_page_no != 0):
                for page_no in range(1, max_page_no+1):

                    try:
                    # time.sleep(1)
                        ESPN_IN_URL = f"https://site.web.api.espn.com/apis/site/v2/sports/cricket/{series_id}/playbyplay?contentorigin=espn&event={match_id}&page={page_no}&period={inn_no}"
                        inn_response = requests.get(ESPN_IN_URL, timeout=100)
                        inn_data = inn_response.json()
                        match_data.append(inn_data['commentary']['items'])
                        print(inn_no, page_no)
                        
                    except requests.exceptions.RequestException as e:
                        continue

        os.makedirs(f"espn.in/{season_no}", exist_ok=True)

        with open(f"espn.in/{season_no}/{match_id}.json", 'w') as f:
            json.dump(match_data, f, indent=4)


In [ ]:
espn_in_comms_data(mid_sid)
# mid_sid

In [19]:
import pandas as pd

init_df = pd.read_json(f"espn.in/seasons/tests_{season_no}.json")


In [20]:
flattened_list = [item for row in init_df.itertuples(index=False) for item in row if item is not None]
espn_in_df = pd.DataFrame(flattened_list)
espn_in_df = espn_in_df.drop(columns=['id', 'clock', 'team', 'athletesInvolved','preText', 'text', 'postText', 'sequence', 'shortText', 'periodText', 'media', 'mediaId'])
espn_in_df.insert(0, 'matchID', match_id)

In [21]:
espn_in_df_new = pd.concat([espn_in_df.drop("playType", axis=1), espn_in_df["playType"].apply(pd.Series).add_prefix('playType_')], axis=1)

espn_in_df_new = pd.concat([espn_in_df_new.drop("bowler", axis=1), espn_in_df_new["bowler"].apply(pd.Series).add_prefix('bowler_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("batsman", axis=1), espn_in_df_new["batsman"].apply(pd.Series).add_prefix('batsman_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("otherBatsman", axis=1), espn_in_df_new["otherBatsman"].apply(pd.Series).add_prefix('otherbatsman_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("otherBowler", axis=1), espn_in_df_new["otherBowler"].apply(pd.Series).add_prefix('otherbowler_')], axis=1)

espn_in_df_new = pd.concat([espn_in_df_new.drop("over", axis=1), espn_in_df_new["over"].apply(pd.Series).add_prefix('over_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("innings", axis=1), espn_in_df_new["innings"].apply(pd.Series).add_prefix('inns_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("dismissal", axis=1), espn_in_df_new["dismissal"].apply(pd.Series).add_prefix('dismissal_')], axis=1)

espn_in_df_new['date'] = pd.to_datetime(espn_in_df_new['date'], format='%Y-%m-%dT%H:%MZ')
espn_in_df_new['match_date'] = espn_in_df_new['date'].dt.date
espn_in_df_new['match_year'] = espn_in_df_new['date'].dt.year

In [ ]:
espn_in_df

In [ ]:
espn_in_df_new = espn_in_df_new.drop(columns=['date'])
espn_in_df_new.columns

In [24]:
espn_in_df_new['bowler_team'] = espn_in_df_new['bowler_team'].apply(lambda x: x['name'] if isinstance(x, dict) else x)
espn_in_df_new['batsman_team'] = espn_in_df_new['batsman_team'].apply(lambda x: x['name'] if isinstance(x, dict) else x)
espn_in_df_new['otherbatsman_team'] = espn_in_df_new['otherbatsman_team'].apply(lambda x: x['name'] if isinstance(x, dict) else x)
espn_in_df_new['otherbowler_team'] = espn_in_df_new['otherbowler_team'].apply(lambda x: x['name'] if isinstance(x, dict) else x)

espn_in_df_new = pd.concat([espn_in_df_new.drop("bowler_athlete", axis=1), espn_in_df_new["bowler_athlete"].apply(pd.Series).add_prefix('bowler_athlete_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("batsman_athlete", axis=1), espn_in_df_new["batsman_athlete"].apply(pd.Series).add_prefix('batsman_athlete_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("otherbatsman_athlete", axis=1), espn_in_df_new["otherbatsman_athlete"].apply(pd.Series).add_prefix('otherbatsman_athlete_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("otherbowler_athlete", axis=1), espn_in_df_new["otherbowler_athlete"].apply(pd.Series).add_prefix('otherbowler_athlete_')], axis=1)

# espn_in_df_new = pd.concat([espn_in_df_new.drop("dismissal_batsman", axis=1), espn_in_df_new["dismissal_batsman"].apply(pd.Series).add_prefix('dismissal_batsman_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("dismissal_fielder", axis=1), espn_in_df_new["dismissal_fielder"].apply(pd.Series).add_prefix('dismissal_fielder_')], axis=1)
espn_in_df_new = pd.concat([espn_in_df_new.drop("dismissal_fielder_athlete", axis=1), espn_in_df_new["dismissal_fielder_athlete"].apply(pd.Series).add_prefix('dismissal_fielder_athlete_')], axis=1)


In [25]:
espn_in_df_new = espn_in_df_new.drop(columns=['bowler_athlete_shortName', 'bowler_athlete_fullName', 'bowler_athlete_displayName',
                                            'otherbowler_athlete_shortName', 'otherbowler_athlete_fullName', 'otherbowler_athlete_displayName',
                                            'batsman_athlete_shortName', 'batsman_athlete_fullName', 'batsman_athlete_displayName',
                                            'otherbatsman_athlete_shortName', 'otherbatsman_athlete_fullName', 'otherbatsman_athlete_displayName',
                                            'dismissal_fielder_athlete_shortName', 'dismissal_fielder_athlete_fullName', 'dismissal_fielder_athlete_displayName',
                                            'dismissal_fielder_0', 'dismissal_fielder_athlete_0', 'dismissal_batsman', 'dismissal_bowler', 'dismissal_text',
                                            'over_balls', 'over_limit', 'over_actual', 'over_overs', 'inns_id', 'inns_number', 'inns_remainingBalls', 'inns_ballLimit',
                                            'dismissal_retiredText'])

In [ ]:
espn_in_df_new.columns

In [27]:
espn_in_df_new.columns = ['matchID', 'innings', 'homeScore', 'awayScore', 'totalruns',
       'bbbTimestamp', 'playType_id', 'playType_outcome', 

       'curr_bowl_team','curr_bowl_maidens', 'curr_bowl_balls', 'curr_bowl_wickets', 'curr_bowl_overs','curr_bowl_conceded', 
       'curr_bat_team', 'curr_bat_runs', 'curr_bat_bf','curr_bat_fours', 'curr_bat_runs', 'curr_bat_sixes', 
       'curr_non_bat_team','curr_non_bat_runs', 'curr_non_bat_bf', 'curr_non_bat_fours','curr_non_bat_runs', 'curr_non_bat_sixes', 
       'curr_non_bowl_team','curr_non_bowl_maidens', 'curr_non_bowl_balls', 'curr_non_bowl_wickets','curr_non_bowl_overs', 'curr_non_bowl_conceded',
         
       'over_ball', 'over_complete', 'over_maiden', 'over_noBall',
       'over_wide', 'over_legByes', 'over_byes', 'over_number', 'over_runs',
       'over_wickets','over_unique',

       'inns_runRate', 'inns_byes',
       'inns_balls', 'inns_noBalls', 'inns_wickets', 'inns_legByes',
       'inns_target', 'inns_session', 'inns_day',
       'inns_fallOfWickets', 'inns_trailBy', 'inns_leadBy',
       'inns_remainingOvers', 'inns_totalRuns', 'inns_wides', 'inns_runs',

       'is_dismissal', 'is_bowled', 'dismissal_type',
       'dismissal_minutes', 'match_date', 'match_year',

       'curr_bowl_id', 'curr_bowl_name', 'curr_bat_id',
       'curr_bat_name', 'curr_non_bat_id',
       'curr_non_bat_name', 'curr_non_bowl_id',
       'curr_non_bowl_name', 'dismissal_isKeeper',
       'dismissal_fielder_id', 'dismissal_fielder_name']

In [28]:
espn_in_df_new = espn_in_df_new[[
        'matchID', 'innings', 'homeScore', 'awayScore', 'totalruns','bbbTimestamp', 'playType_id', 'playType_outcome', 

       'curr_bat_id', 'curr_bat_name','curr_bat_team', 'curr_bat_runs', 'curr_bat_bf','curr_bat_fours', 'curr_bat_runs', 'curr_bat_sixes', 
       'curr_non_bat_id','curr_non_bat_name', 'curr_non_bat_team','curr_non_bat_runs', 'curr_non_bat_bf', 'curr_non_bat_fours','curr_non_bat_runs', 'curr_non_bat_sixes', 

       'curr_bowl_id', 'curr_bowl_name', 'curr_bowl_team','curr_bowl_maidens', 'curr_bowl_balls', 'curr_bowl_wickets', 'curr_bowl_overs','curr_bowl_conceded',
       'curr_non_bowl_id', 'curr_non_bowl_name', 'curr_non_bowl_team','curr_non_bowl_maidens', 'curr_non_bowl_balls', 'curr_non_bowl_wickets','curr_non_bowl_overs', 'curr_non_bowl_conceded',
         
       'over_ball', 'over_complete', 'over_maiden', 'over_noBall',
       'over_wide', 'over_legByes', 'over_byes', 'over_number', 'over_runs',
       'over_wickets', 'over_unique',

       'inns_runRate', 'inns_byes',
       'inns_balls', 'inns_noBalls', 'inns_wickets', 'inns_legByes',
       'inns_target', 'inns_session', 'inns_day',
       'inns_fallOfWickets', 'inns_trailBy', 'inns_leadBy',
       'inns_remainingOvers', 'inns_totalRuns', 'inns_wides', 'inns_runs',

       'is_dismissal', 'is_bowled', 'dismissal_isKeeper', 'dismissal_type', 'dismissal_fielder_id', 'dismissal_fielder_name','dismissal_minutes', 
       'match_date', 'match_year']]

In [29]:
espn_in_df_new['curr_bowl_overs'] = espn_in_df_new['curr_bowl_overs'].round(1)
espn_in_df_new['curr_non_bowl_overs'] = espn_in_df_new['curr_non_bowl_overs'].round(1)
espn_in_df_new['inns_runRate'] = espn_in_df_new['inns_runRate'].round(2)

In [30]:
espn_in_df_new.to_csv(f"./espn.in/{match_id}.csv", index=False)